In [36]:
pip install scikit-learn==1.2.0

Note: you may need to restart the kernel to use updated packages.


In [38]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

In [39]:
def predict(input):
    model_dict = pickle.load(open('./model.p', 'rb'))
    model = model_dict['model']
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles

    hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)
    labels_dict = {0: 'L', 1: 'A', 2: 'B', 3: 'c', 4: 'Hello'}

#while True:
    data_aux = []
    frame = cv2.imread(input)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame_rgb)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
            frame,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
        

        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y
            data_aux.append(x)
            data_aux.append(y)
    
    if data_aux:
        prediction = model.predict([np.asarray(data_aux)])
        predicted_character = labels_dict[int(prediction[0])]
        return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), predicted_character
        # Process the prediction as needed
    else:
        return "No hand landmarks detected in the current frame."
         

In [47]:
import gradio as gr

demo = gr.Interface(fn=predict,
             inputs=gr.Image(label="camera input", type="filepath", show_label=True), 
             outputs = ["image", "text"], title = " Sign translation App")
demo.launch()

Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\rajak\anaconda3\Lib\site-packages\gradio\queueing.py", line 528, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rajak\anaconda3\Lib\site-packages\gradio\route_utils.py", line 270, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rajak\anaconda3\Lib\site-packages\gradio\blocks.py", line 1908, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rajak\anaconda3\Lib\site-packages\gradio\blocks.py", line 1485, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rajak\anaconda3\Lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^